<a href="https://colab.research.google.com/github/YasuharuSuzuki/25_isco3/blob/main/" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 03_RandomForestによる分類モデルの学習
RandomForestを使って分類モデルを作り、SVCと比べてみましょう。  
また、データ拡張を行って、不均衡データのデータ数を揃えて学習しましょう。  

---

### 🌐 参考サイト
#### [【Python入門】7-2. scikit-learnライブラリ](https://utokyo-ipp.github.io/7/7-2.html)  
#### [imbalanced-learnを利用してアンダー/オーバーサンプリングを実施してみた](https://zenn.dev/akasan/articles/7a148787cb3be8)  

---

### ⬇️ ライブラリのインストール

In [ ]:
# matplotlib日本語対応ライブラリのインストール
!pip install matplotlib-fontja

### 📁 GoogleDrive の Mount

In [ ]:
# drive.mount()を使用してGoogleDriveをMount
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# os.chdir()を使用してカレントディレクトリを変更。作成したショートカット名を使ってパスを指定します。
import os
os.chdir('/content/drive/MyDrive/＜作成したショートカット名＞')

### ➜]　ライブラリのimport

In [2]:
import pandas as pd # pandasモジュールを読み込む（データ分析ライブラリ）
import numpy as np # NumPyモジュールを読み込む（数値演算ライブラリ）
import matplotlib.pyplot as plt # matplotlib.pyplot モジュールを読み込む（可視化ライブラリ。細かいカスタマイズが可能）
import seaborn as sns # Seabornモジュールを読み込む（可視化ライブラリ。pairplotなど多くのデータ分析情報をplotするのに向いている）
import joblib # モデル保存・読み込みライブラリ

from sklearn import svm # scikit-learnのSVM(サポートベクターマシン)
from sklearn.model_selection import train_test_split # scikit-learnの訓練データ / テストデータ 分割ライブラリ
from sklearn import metrics # scikit-learnの精度算出用ライブラリ
from sklearn.ensemble import RandomForestClassifier # scikit-learnのRandomForest(ランダムフォレスト)

from imblearn.over_sampling import SMOTE  # 不均衡データサンプリングライブラリ
from mpl_toolkits.axes_grid1 import Divider, Size # matplotlib 領域修正用クラス
from mpl_toolkits.axes_grid1.mpl_axes import Axes # matplotlib 領域修正用クラス

### ✍️ 定数の定義
- データファイルパスを定義しておきます
- データ読み込みの際にご使用してください。修正は不要です

In [15]:
# データファイルパス定義
WINE_CSV = "./data/wine.csv"
IRIS_CSV = "./data/iris.csv"

# モデルファイルパス定義
WINE_RF_MODEL = "./data/wine_rf.joblib"
IRIS_RF_MODEL = "./data/iris_rf.joblib"

# RANDOM SEED 定義
RANDOM_SEED = 0

### 🍸 サンプルプログラム3 ワイン分類モデルの学習[RandomForest]

以下の手順で、ワインの分類モデルを学習します。
- ワインのデータセットを読み込む
- 読み込んだDataFrameを確認する
- SMOTEを使ってデータ拡張を行う
- RandomForestを用いて分類モデルを学習する
- 学習したモデルをjoblib.dump()でファイルに保存する(WINE_RF_MODELで定義済みのファイル名を使用する)

In [16]:
# ワインのデータセットを読み込み
df_wine = pd.read_csv(WINE_CSV, index_col=None) # indexに使用するカラムが無いため、index_col=Noneを指定します

# 読み込んだDataFrameを確認
df_wine.head()

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline,target
0,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065,1
1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050,1
2,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185,1
3,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480,1
4,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735,1


In [17]:
# 説明変数と目的変数に分ける
X = df_wine.iloc[:,:-1]
y = df_wine.iloc[:,-1]

# 訓練データとテストデータに分割する
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=RANDOM_SEED) # 特に理由がない限りは、学習データ：テストデータは 7：3 でわけましょう。
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=RANDOM_SEED, stratify=y)  # stratifyを指定すると層別にきれいに分けることができます

In [18]:
# SMOTEを使用して不均衡データを水増し（データ拡張）する
# 揃える数を決める。品種ごとのn数のうち、最大の値に揃えるようにdictを作成する
value_counts = y_train.value_counts()
max_count = max(y_train.value_counts().values)
smote_ratio = {index : max_count for index, value in value_counts.items()}
print("smote_ratio =", smote_ratio)

# SMOTEでデータ拡張を実施
smote = SMOTE(sampling_strategy=smote_ratio, random_state=0)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

# 揃えたあとのn数を確認
y_train_resampled.value_counts()

smote_ratio = {2: np.int64(49), 1: np.int64(49), 3: np.int64(49)}


target
1    49
2    49
3    49
Name: count, dtype: int64

In [19]:
# RandomForestで学習する
rf_model = RandomForestClassifier(max_depth=3)    # max_depth のパラメータを指定する
rf_model.fit(X_train_resampled, y_train_resampled)     # 訓練データと教師データで学習する

# 学習したモデルを保存する
joblib.dump(rf_model, WINE_RF_MODEL)

['./data/wine_rf.joblib']

### 🪻 練習プログラム3 あやめ分類モデルの学習[RandomForest] (5点)

以下の手順で、あやめの分類モデルを学習してみましょう。  
1️⃣ あやめのデータセットを読み込む   
2️⃣ 読み込んだDataFrameを確認する  
3️⃣ SMOTEを使ってデータ拡張を行う  
4️⃣ RandomForestを用いて分類モデルを学習する   
5️⃣ 学習したモデルをjoblib.dump()でファイルに保存する(IRIS_RF_MODELで定義済みのファイル名を使用する)   

<!-- PROBLEM_TYPE: SYNTAX_EXERCISE -->

In [20]:
# あやめのデータセットを読み込み
df_iris = pd.read_csv(IRIS_CSV, index_col=0) # idをindexに使用するため、index_col=0を指定します

# 読み込んだDataFrameを確認
df_iris.head()

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
Id,,,,,
1,5.1,3.5,1.4,0.2,Iris-setosa
2,4.9,3.0,1.4,0.2,Iris-setosa
3,4.7,3.2,1.3,0.2,Iris-setosa
4,4.6,3.1,1.5,0.2,Iris-setosa
5,5.0,3.6,1.4,0.2,Iris-setosa


In [21]:
# 説明変数と目的変数に分ける
X = df_iris.iloc[:,:-1]
y = df_iris.iloc[:,-1]

# 訓練データとテストデータに分割する
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=RANDOM_SEED) # 特に理由がない限りは、学習データ：テストデータは 7：3 でわけましょう。
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=RANDOM_SEED, stratify=y)  # stratifyを指定すると層別にきれいに分けることができます

In [22]:
# SMOTEを使用して不均衡データを水増し（データ拡張）する
# 揃える数を決める。品種ごとのn数のうち、最大の値に揃えるようにdictを作成する
value_counts = y_train.value_counts()
max_count = max(y_train.value_counts().values)
smote_ratio = {index : max_count for index, value in value_counts.items()}
print("smote_ratio =", smote_ratio)

# SMOTEでデータ拡張を実施
smote = SMOTE(sampling_strategy=smote_ratio, random_state=0)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

# 揃えたあとのn数を確認
y_train_resampled.value_counts()

smote_ratio = {'Iris-virginica': np.int64(39), 'Iris-setosa': np.int64(39), 'Iris-versicolor': np.int64(39)}


Species
Iris-versicolor    39
Iris-virginica     39
Iris-setosa        39
Name: count, dtype: int64

In [23]:
# RandomForestで学習する
rf_model = RandomForestClassifier(max_depth=3)    # max_depth のパラメータを指定する
rf_model.fit(X_train_resampled, y_train_resampled)     # 訓練データと教師データで学習する

# 学習したモデルを保存する
joblib.dump(rf_model, IRIS_RF_MODEL)

['./data/iris_rf.joblib']

In [ ]:
#@title 送信処理用共通プログラム（以下のセルを「▶ 実行」して下さい。） { display-mode: "form" }
#@markdown 解答の送信に必要です。（中身は絶対に修正しないでください）

import os
os.makedirs(".client/python", exist_ok=True)  # 隠しフォルダを作成
!wget -q https://raw.githubusercontent.com/YasuharuSuzuki/25_isco3/main/91_notebook_client/src/setup.sh -O .client/setup.sh   # 外部コードをダウンロードして隠しフォルダに配置
!chmod +x ./.client/setup.sh
!bash ./.client/setup.sh # 実行

# sandbox実行用
# os.symlink("../../src", ".client", target_is_directory=True)
# !bash ./sandbox_setup.sh # 実行

# モジュールの初期化
exec(open('.client/client_setup.py').read())

# ノートブック固有の設定
NOTEBOOK_PATH = "04_機械学習(分類)/03_RandomForestによる分類モデルの学習.ipynb"
set_notebook_config(NOTEBOOK_PATH)

In [ ]:
#@title 問3 送信ボタン表示（以下のセルを「▶ 実行」して下さい。） { display-mode: "form" }
#@markdown メールアドレスを取得して、送信ボタンを押してください（中のプログラムは絶対に修正しないでください）
display(create_submit_button(problem_number=3, button_name="練習プログラム"))